In [1]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
glove_file = datapath('/home/mounika/Desktop/Word-Embeddings/dranziera_protocol/glove.42B.300d.txt')
tmp_file = get_tmpfile("glove.42B.300d_word2vec.txt")
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_file, tmp_file)
model = KeyedVectors.load_word2vec_format(tmp_file)

In [1]:
import pandas as pd
import numpy as np

In [3]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [4]:
def glove_features(data):
    x_hk_w2 = []
    for i in data:
        tokens = nlp(i)
        temp = []
        for token in tokens:
            try:
                temp.append(model[str(token).lower()])
            except:
                continue
        temp = np.array(temp)
        if temp.size==0:
            temp = np.zeros([1,300])
        #print(temp.shape)
        x_hk_w2.append(np.average(temp,axis=0))
    x_hk_w2 = np.array(x_hk_w2)
    print(x_hk_w2.shape)
    return x_hk_w2

In [25]:
fp = open('./Video_Games/OnlyText.txt','r')
lines = fp.readlines()

In [26]:
text_data = []
for line in lines:
    text_data.append(line.strip('\n'))

In [27]:
print(len(text_data))

50000


In [28]:
x_features_hk_glove = glove_features(text_data)

(50000, 300)


In [29]:
np.save('/home/mounika/Desktop/Word-Embeddings/dranziera_protocol/GLOVE_features/Video_Games_glove.npy',x_features_hk_glove)

In [257]:
x_features = np.load('./GLOVE_features/Video_Games_glove.npy')

In [258]:
labels = pd.read_csv('./Video_Games/polarity.txt',header=None)

In [259]:
labels.shape

(50000, 1)

In [260]:
y = []
for i in labels[0]:
    if i=='positive':
        y.append(1)
    else:
        y.append(0)

In [261]:
y = np.array(y)

In [262]:
print(x_features.shape)
print(y.shape)

(50000, 300)
(50000,)


In [263]:
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import classification_report,confusion_matrix
x_train,x_test,y_train,y_test = train_test_split(x_features,y,test_size=0.2,random_state=47)

In [264]:
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
import argparse

In [265]:

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--model", dest="model", type=str, default=None, help="gcfoest Net Model File")
    args = parser.parse_args()
    return args


def get_toy_config():
    config = {}
    ca_config = {}
    ca_config["random_state"] = 0
    ca_config["max_layers"] = 100
    ca_config["early_stopping_rounds"] = 3
    ca_config["n_classes"] = 2
    ca_config["estimators"] = []
    ca_config["estimators"].append(
            {"n_folds": 5, "type": "LGBClassifier", "n_estimators": 10, "max_depth": 5,
             "objective": "binary", "silent": True, "nthread": 5, "learning_rate": 0.1} )
    ca_config["estimators"].append({"n_folds": 5, "type": "RandomForestClassifier", "n_estimators": 10, "max_depth": None, "n_jobs": -1})
    ca_config["estimators"].append({"n_folds": 5, "type": "ExtraTreesClassifier", "n_estimators": 10, "max_depth": None, "n_jobs": -1})
    ca_config["estimators"].append({"n_folds": 5, "type": "LogisticRegression"}),
    #ca_config["estimators"].append({"n_folds": 5, "type": "GradientBoostingClassifier"})
    config["cascade"] = ca_config
    return config

In [266]:
config = get_toy_config()
gc = GCForest(config)

In [267]:
gc.config

{'cascade': {'random_state': 0,
  'max_layers': 100,
  'early_stopping_rounds': 3,
  'n_classes': 2,
  'estimators': [{'n_folds': 5,
    'type': 'LGBClassifier',
    'n_estimators': 10,
    'max_depth': 5,
    'objective': 'binary',
    'silent': True,
    'nthread': 5,
    'learning_rate': 0.1},
   {'n_folds': 5,
    'type': 'RandomForestClassifier',
    'n_estimators': 10,
    'max_depth': None,
    'n_jobs': -1},
   {'n_folds': 5,
    'type': 'ExtraTreesClassifier',
    'n_estimators': 10,
    'max_depth': None,
    'n_jobs': -1},
   {'n_folds': 5, 'type': 'LogisticRegression'}]}}

In [268]:
X_train_enc = gc.fit_transform(x_train, y_train,X_test=x_test, y_test=y_test)
#y_pred = gc.predict(x_test)

[ 2019-03-27 20:02:39,278][cascade_classifier.fit_transform] X_groups_train.shape=[(40000, 300)],y_train.shape=(40000,),X_groups_test.shape=[(10000, 300)],y_test.shape=(10000,)
[ 2019-03-27 20:02:39,322][cascade_classifier.fit_transform] group_dims=[300]
[ 2019-03-27 20:02:39,323][cascade_classifier.fit_transform] group_starts=[0]
[ 2019-03-27 20:02:39,323][cascade_classifier.fit_transform] group_ends=[300]
[ 2019-03-27 20:02:39,324][cascade_classifier.fit_transform] X_train.shape=(40000, 300),X_test.shape=(10000, 300)
[ 2019-03-27 20:02:39,368][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(40000, 300), X_cur_test.shape=(10000, 300)
[ 2019-03-27 20:02:40,107][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 5_folds.train_0.predict)=74.63%
[ 2019-03-27 20:02:40,831][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 5_folds.train_1.predict)=74.30%
[ 2019-03-27 20:02:41,544][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 -

[ 2019-03-27 20:03:21,991][kfold_wrapper.log_eval_metrics] Accuracy(layer_1 - estimator_3 - 5_folds.train_0.predict)=83.65%
/home/mounika/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
[ 2019-03-27 20:03:23,817][kfold_wrapper.log_eval_metrics] Accuracy(layer_1 - estimator_3 - 5_folds.train_1.predict)=82.91%
/home/mounika/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
[ 2019-03-27 20:03:25,959][kfold_wrapper.log_eval_metrics] Accuracy(layer_1 - estimator_3 - 5_folds.train_2.predict)=83.97%
/home/mounika/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.


[ 2019-03-27 20:04:05,224][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_1 - 5_folds.train_3.predict)=82.56%
[ 2019-03-27 20:04:06,732][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_1 - 5_folds.train_4.predict)=81.97%
[ 2019-03-27 20:04:06,852][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_1 - 5_folds.train_cv.predict)=82.09%
[ 2019-03-27 20:04:06,855][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_1 - 5_folds.test.predict)=82.94%
[ 2019-03-27 20:04:07,557][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_2 - 5_folds.train_0.predict)=82.30%
[ 2019-03-27 20:04:08,390][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_2 - 5_folds.train_1.predict)=81.81%
[ 2019-03-27 20:04:09,212][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_2 - 5_folds.train_2.predict)=82.34%
[ 2019-03-27 20:04:10,061][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_2 - 5_folds.train_3.predict)=82.27%
[ 2019-03-

[ 2019-03-27 20:04:46,838][cascade_classifier.calc_accuracy] Accuracy(layer_4 - train.classifier_average)=83.78%
[ 2019-03-27 20:04:46,839][cascade_classifier.calc_accuracy] Accuracy(layer_4 - test.classifier_average)=83.34%
[ 2019-03-27 20:04:46,897][cascade_classifier.fit_transform] [layer=5] look_indexs=[0], X_cur_train.shape=(40000, 308), X_cur_test.shape=(10000, 308)
[ 2019-03-27 20:04:47,605][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 5_folds.train_0.predict)=83.98%
[ 2019-03-27 20:04:48,308][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 5_folds.train_1.predict)=82.94%
[ 2019-03-27 20:04:49,014][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 5_folds.train_2.predict)=83.99%
[ 2019-03-27 20:04:49,724][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 5_folds.train_3.predict)=83.44%
[ 2019-03-27 20:04:50,437][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 5_folds.train_4.predict)=84.10%
[ 201

/home/mounika/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
[ 2019-03-27 20:05:31,558][kfold_wrapper.log_eval_metrics] Accuracy(layer_6 - estimator_3 - 5_folds.train_1.predict)=83.69%
/home/mounika/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
[ 2019-03-27 20:05:33,665][kfold_wrapper.log_eval_metrics] Accuracy(layer_6 - estimator_3 - 5_folds.train_2.predict)=82.66%
/home/mounika/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
[ 2019-03-27 20:05:35,492][kfold_wrapper.log_eval_metrics] Accuracy(layer_6 - estimator_3 - 5_folds.train_3

In [269]:
y_pred = gc.predict(x_test)

[ 2019-03-27 20:06:03,663][cascade_classifier.transform] X_groups_test.shape=[(10000, 300)]
[ 2019-03-27 20:06:03,668][cascade_classifier.transform] group_dims=[300]
[ 2019-03-27 20:06:03,669][cascade_classifier.transform] X_test.shape=(10000, 300)
[ 2019-03-27 20:06:03,673][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(10000, 300)
[ 2019-03-27 20:06:04,972][cascade_classifier.transform] [layer=1] look_indexs=[0], X_cur_test.shape=(10000, 308)
[ 2019-03-27 20:06:06,287][cascade_classifier.transform] [layer=2] look_indexs=[0], X_cur_test.shape=(10000, 308)
[ 2019-03-27 20:06:07,587][cascade_classifier.transform] [layer=3] look_indexs=[0], X_cur_test.shape=(10000, 308)
[ 2019-03-27 20:06:08,935][cascade_classifier.transform] [layer=4] look_indexs=[0], X_cur_test.shape=(10000, 308)


In [270]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.82      0.85      0.84      5003
           1       0.85      0.81      0.83      4997

   micro avg       0.83      0.83      0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000



In [271]:
import pickle
pickle.dump(gc,open('/home/mounika/Desktop/Word-Embeddings/dranziera_protocol/GC_GLOVE/gc_glove_Video_Games','wb'))